### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5239,2023-10-24,Eua Nba,20:30,Denver Nuggets,Los Angeles Lakers,1.45,2.80,227.5,1.91,1.91,-5.5,1.91,1.91
5240,2023-10-24,Europa Eurocopa,13:00,BC Wolves,Besiktas,2.05,1.80,157.5,1.91,1.91,1.5,1.91,1.91
5241,2023-10-24,Europa Eurocopa,14:30,Hamburg,Hapoel Tel-Aviv,8.50,1.08,173.5,1.95,1.87,10.5,1.91,1.91
5242,2023-10-24,Europa Eurocopa,15:00,Venezia,Joventut,1.80,2.05,160.5,1.91,1.91,-1.5,1.91,1.91
5243,2023-10-24,Europa Eurocopa,15:30,London Lions,Paris,2.75,1.48,173.5,1.91,1.91,4.5,1.95,1.87
5244,2023-10-24,China Cba,08:35,Beijing,Zhejiang Guangsha,1.67,2.20,190.5,1.87,1.87,-2.5,1.87,1.87
5245,2023-10-24,China Cba,08:35,Liaoning,Sichuan,1.01,19.00,190.5,1.87,1.87,-21.5,1.80,1.95
5246,2023-10-24,China Cba,08:35,Zhejiang Chouzhou,Guangdong,1.45,2.75,202.5,1.87,1.87,-4.5,1.83,1.91
5247,2023-10-24,Coreia Do Sul Kbl,07:00,Goyang Sono,Seoul Knights,3.80,1.28,165.5,1.91,1.91,7.5,1.95,1.87
5248,2023-10-24,Europa Liga Dos Campeões,13:00,Darussafaka,Tenerife,2.40,1.53,157.5,1.83,1.83,3.5,1.83,1.83


## Apostas do dia

### Over v1 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:30,Eua Nba,Denver Nuggets,Los Angeles Lakers,227.5,1.91,0.7936,Over
1,13:00,Europa Eurocopa,BC Wolves,Besiktas,157.5,1.91,0.8383,Over
2,14:30,Europa Eurocopa,Hamburg,Hapoel Tel-Aviv,173.5,1.95,0.8299,Over
3,15:00,Europa Eurocopa,Venezia,Joventut,160.5,1.91,0.8415,Over
4,15:30,Europa Eurocopa,London Lions,Paris,173.5,1.91,0.7127,Over
8,07:00,Coreia Do Sul Kbl,Goyang Sono,Seoul Knights,165.5,1.91,0.7365,Over


### Over v1 (LDA)

In [9]:
# Duplicando o dataset
df_over_v1 = jogos.copy()

df_over_v1.reset_index(drop=True, inplace=True)
df_over_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v1.empty:
    mdl_over_v1 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v1, df_over_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,15:30,Europa Eurocopa,London Lions,Paris,173.5,1.91,0.7127,Over


### Over v2 (LDA)

In [10]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,15:30,Europa Eurocopa,London Lions,Paris,173.5,1.91,0.5846,Over


### Over v3 (Blend)

In [11]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['HA_Line'] < -8.5) | (prev['HA_Line'] > -5.5))]
    prev = prev[((prev['CV_ML'] >= 0) & (prev['CV_ML'] < 0.65)) | ((prev['CV_ML'] >= 0.77) & (prev['CV_ML'] < 0.87)) | ((prev['CV_ML'] >= 0.95) & (prev['CV_ML'] < 1.5))]


    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (BLEND) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (BLEND) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,Bet
2,14:30,Europa Eurocopa,Hamburg,Hapoel Tel-Aviv,173.5,1.95,Over
4,15:30,Europa Eurocopa,London Lions,Paris,173.5,1.91,Over
